# Redes Neuronales

Breve explicacion

## Configuraciones Iniciales

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

import keras

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import redesNeuronalesPreprocessing
from preprocessing import prepararSetDeValidacion

## Carga y preparacion del set de datos

In [ ]:
y = pd.read_csv('Datasets/tp-2020-2c-train-cols1.csv')

In [ ]:
X = pd.read_csv('Datasets/tp-2020-2c-train-cols2.csv')

In [ ]:
X = prepararSetDeDatos(X)
y = prepararSetDeValidacion(y)

## Funciones Auxiliares


In [ ]:
def obtenerMejoresHiperparametros(datosPreprocesados):
    mejor_optimizador = None
    mejor_valor = 0
    mejor_optimizador = None
    mejor_activacion = None
    y_array=np.array(y)
    for lr in [0.01,0.1,0.2,0.3,0.5]:
        for optimizador in [keras.optimizers.Adam(learning_rate=lr),keras.optimizers.SGD(learning_rate=lr)]:
            for activacion in ['relu','tanh','sigmoid']:
                kf = StratifiedKFold(n_splits=3)
                metricas = []
                for fold_idx, (train_index, test_index) in enumerate(kf.split(datosPreprocesados, y_array)):
                    es = EarlyStopping(monitor='val_acc', mode='max')
                    
                    primera_capa = keras.layers.Input(shape=(14,))
                    capa_intermedia = keras.layers.Dense(15, activation=activacion,kernel_regularizer=regularizers.l2(0.001))(primera_capa)
                    dropout = keras.layers.Dropout(0.25)(capa_intermedia)
                    ultima_capa = keras.layers.Dense(1, activation='softmax',kernel_regularizer=regularizers.l2(0.001))(dropout)
                    red_neuronal = keras.models.Model(inputs=[primera_capa], outputs=[ultima_capa])

                    red_neuronal.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer = optimizador)

                    red_neuronal.fit(datosPreprocesados[train_index], y_array[train_index].ravel(), epochs=350, batch_size=150, verbose=0, validation_split=0.25)

                    predicciones = red_neuronal.predict(datosPreprocesados[test_index])
                    score_obtenida = roc_auc_score(y_array[test_index],predicciones)
                    metricas.append(score_obtenida)

                if np.mean(metricas) >= mejor_valor:
                    mejor_valor = np.mean(metricas)
                    mejor_optimizador = optimizador
                    mejor_activacion = activacion
            
    return mejor_valor, mejor_optimizador, mejor_activacion

## Redes Neuronales

In [ ]:
X_redes_neuronales = redesNeuronalesPreprocessing(X)

In [ ]:
mejor_valor, mejor_optimizador, mejor_activacion = obtenerMejoresHiperparametros(X_redes_neuronales) # preguntar como conviene obtener la mejor configuracion

In [ ]:
print(mejor_valor.round(3))
print(mejor_optimizador)
print(mejor_activacion)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_redes_neuronales, y, test_size=0.25, random_state=0)

Armamos la red neuronal ahora con la mejor combinacion de hiperparametros encontrados.

In [ ]:
primera_capa = keras.layers.Input(shape=(14,))
capa_intermedia = keras.layers.Dense(25, activation='relu')(primera_capa)
capa_intermedia2 = keras.layers.Dense(20, activation='tanh')(capa_intermedia)
capa_intermedia3 = keras.layers.Dense(20, activation='relu')(capa_intermedia2)
ultima_capa = keras.layers.Dense(1, activation='sigmoid')(capa_intermedia3)

red_neuronal = keras.models.Model(inputs=[primera_capa], outputs=[ultima_capa])

In [ ]:
red_neuronal.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer = mejor_optimizador)

Mostramos el resumen de como queda armada.

In [ ]:
red_neuronal.summary()

Ahora finalmente entrenamos con el set de entrenamiento.

In [ ]:
h = red_neuronal.fit(X_train, y_train, epochs=500, batch_size=100, verbose=0, validation_split=0.25)

Observamos como se fue desarrollando la funcion de perdida para el entrenamiento y la validacion de la red.

In [ ]:
plt.figure(dpi=125, figsize=(12, 4))
plt.plot(h.history['loss'], label="training loss")
plt.plot(h.history['val_loss'], label="validation loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

Ahora buscamos las metricas que nos interesan sobre el set de evaluacion guardado anteriormente.

In [ ]:
y_pred = red_neuronal.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
fpr_red_test,tpr_red_test,thresholds_red_test = roc_curve(y_test,red_neuronal.predict_proba(X_test)[:,1])

zero_test = np.argmin(np.abs(thresholds_red_test))

plt.plot(fpr_red_test,tpr_red_test,label="ROC Curve KNN Test")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.plot(fpr_red_test[zero_test],tpr_red_test[zero_test],'o',markersize=10,label="threshold zero test",fillstyle="none",c="k",mew=2)

plt.legend(loc=4)
plt.show()

In [ ]:
auc_red = roc_auc_score(y_test,red_neuronal.predict_proba(X_test)[:, 1])
print("AUC para redes neuronales: {:.3f}".format(auc_red))

## Conclusiones